In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 46.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
# Cấu hình quantization (tùy chọn)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [ ]:
# Tải model và tokenizer
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # load_in_4bit=True,
    quantization_config=quantization_config,
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00017-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Prompt cho summarization
text_to_summarize = """
Natural Language Processing (NLP) is a field of artificial intelligence that focuses on the interaction between computers and humans using natural language. It enables machines to understand, interpret, and generate human language in a valuable way. Key applications include machine translation, sentiment analysis, and text summarization.
"""
messages = [
    {"role": "user", "content": f"Summarize the following text in 1 sentence: {text_to_summarize}"}
]

# Generate
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
outputs = model.generate(inputs, max_new_tokens=150, do_sample=False)  # do_sample=False cho deterministic output

# Decode
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[INST] Summarize the following text in 1 sentence: 
Natural Language Processing (NLP) is a field of artificial intelligence that focuses on the interaction between computers and humans using natural language. It enables machines to understand, interpret, and generate human language in a valuable way. Key applications include machine translation, sentiment analysis, and text summarization.
 [/INST] Natural Language Processing is a subset of artificial intelligence that focuses on enabling machines to understand, interpret, and generate human language effectively, with applications in areas such as machine translation, sentiment analysis, and text summarization.


In [ ]:
# Prompt cho dịch máy đa ngôn ngữ
text = "Natural Language Processing is advancing rapidly with MoE models."
messages = [
    {"role": "user", "content": f"Translate the following text from English to Vietnamese, keeping technical terms accurate: '{text}'"}
]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
outputs = model.generate(inputs, max_new_tokens=200, temperature=0.5)

translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(translated)  # Kết quả mẫu: "Xử Lý Ngôn Ngữ Tự Nhiên đang phát triển nhanh chóng với các mô hình MoE."

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Translate the following text from English to Vietnamese, keeping technical terms accurate: 'Natural Language Processing is advancing rapidly with MoE models.' [/INST] Xử lý Ngôn ngữ Tự nhiên đang phát triển rất nhanh với các mô hình MoE (Mixture of Experts).

Here's the explanation for the technical term MoE (Mixture of Experts):

Trong lĩnh vực Xử lý Ngôn ngữ Tự nhiên, MoE (Mixture of Experts) là một kiểu mô hình phân tích dự đoán có khả năng tùy biến và tính linh hoạt cao. Nó bao gồm một tập hợp các mô hình học máy (hoặc "chuyên gia") được 
